## ANN

In [2]:
import sys
import os

import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import auc, roc_curve

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import ensemble
from scipy.spatial.distance import pdist, squareform
from sklearn.model_selection import cross_val_score, ShuffleSplit


import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
matplotlib.rcParams['font.sans-serif'] = ['FangSong'] # 指定默认字体
matplotlib.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

### ANN without 1h/h average SBP & DBP

In [20]:
import torch
data_range_pro = pd.read_csv('/content/drive/My Drive/hypertension test/Nocturnal_hypertension by C/data_10factors_gbk_dayBP_201222.csv', encoding = 'gbk', index_col = 0)
def preprocess(data0):
    data = data0.copy()
    X_col = [col for col in data.columns if col != 'nocturnal_hypertension']
    dummy_col = [col for col in X_col if len(data[col].unique()) < 5]
    data[dummy_col] = data[dummy_col].astype('int64')
    float_col = [col for col in X_col if col not in dummy_col]
    # standard
    m,s = data[float_col].mean(), data[float_col].std()
    for col in float_col:
        data[col] = (data[col] - data[col].mean()) / (data[col].std())
    temp = pd.DataFrame()
    for col in dummy_col:
        temp = pd.concat([temp, pd.get_dummies(data[col], prefix = col, prefix_sep = '_', drop_first = True)], axis = 1)

    temp = pd.concat([data[float_col], temp, data['nocturnal_hypertension']], axis = 1)
    temp.index = pd.Index(range(len(temp)))
    return temp, m, s, float_col, dummy_col, X_col, 'nocturnal_hypertension'

data, m, s, float_col, dummy_col, X_col, y_col = preprocess(data_range_pro)
data['nocturnal_hypertension'] = data['nocturnal_hypertension'].astype('int')#bool转换成int

R = 115306

temp = ['Clinic SBP,mmHg','eGFR,mL/ (min·1.73 m2)','BUN,mmol/L','Clinic DBP,mmHg'
             ,'nRAAS drugs intake_1', 'Hypertension_1','Age,y'] + [y_col]

data_tensor = torch.from_numpy(data[temp].values).float()

X = data_tensor[:,:-1]
y = data_tensor[:,-1].resize(3103,1)

kf = KFold(10,shuffle = True, random_state = R)

result = []
for train_idx, test_idx in kf.split(range(X.shape[0])):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    model = torch.nn.Sequential(torch.nn.Linear(7,15),
                            torch.nn.Sigmoid(), 
                            torch.nn.Linear(15,1),
                            torch.nn.Sigmoid())
    loss = torch.nn.MSELoss(reduce = False)
    optimizer=torch.optim.SGD(model.parameters(),lr=0.05,weight_decay=0.001)
    sw = y_train.clone()

    sw[sw == 1] = y_train.shape[0]/(2*y_train.float().sum())   
    
    sw[sw == 0] = y_train.shape[0]/(2*(y_train==0).float().sum())
    
    for epoch in range(10000):
        optimizer.zero_grad()  
        out = model(X_train)  
        loss1 = (sw * loss(out, y_train)).sum()/sw.sum()  
        loss1.backward()    
        optimizer.step()    
    out_train = model(X_train) 
    out_test = model(X_test)

    acc_train = ((out_train>0.5).float()==y_train).float().mean()
    acc_test = ((out_test>0.5).float()==y_test).float().mean()
    sen_train =  ((out_train >0.5).float() == y_train)[y_train == 1].float().mean()
    sen_test =  ((out_test >0.5).float() == y_test)[y_test == 1].float().mean()
    spe_train =  ((out_train >0.5).float() == y_train)[y_train == 0].float().mean()
    spe_test =  ((out_test >0.5).float() == y_test)[y_test == 0].float().mean()

    train_fpr, train_tpr, train_thresholds = roc_curve(y_train.detach().numpy(), out_train.detach().numpy())
    test_fpr, test_tpr, test_thresholds = roc_curve(y_test.detach().numpy(), out_test.detach().numpy())
    train_auc = auc(train_fpr, train_tpr)
    test_auc = auc(test_fpr, test_tpr)

    train_pre = ((out_train>0.5)&(y_train == 1)).sum().numpy()/(out_train >0.5).sum().numpy()
    test_pre = ((out_test>0.5)&(y_test == 1)).sum().numpy()/(out_test>0.5).sum().numpy()

    F1_train = (2*train_pre*sen_train)/(train_pre + sen_train)
    F1_test = (2*test_pre*sen_test)/(test_pre + sen_test)

    result.append({'train':{'acc':acc_train, 'sensitivity':sen_train, 'specivity':spe_train,'auc':train_auc, 'F1':F1_train},
                  'test':{'acc':acc_test, 'sensitivity':sen_test, 'specivity':spe_test,'auc':test_auc, 'F1':F1_test}})
acc_train, acc_test, sen_train, sen_test, spe_train, spe_test, auc_train, auc_test,F1_train, F1_test = 0,0,0,0,0,0,0,0,0,0
for temp in result:
    a =  temp['train']
    b = temp ['test']
    acc_train += 0.1*a['acc']
    acc_test += 0.1*b['acc']
    sen_train += 0.1*a['sensitivity']
    sen_test += 0.1*b['sensitivity']
    spe_train += 0.1*a['specivity']
    spe_test += 0.1*b['specivity']
    auc_train += 0.1*a['auc']
    auc_test += 0.1*b['auc']
    F1_train += 0.1*a['F1']
    F1_test += 0.1 * b['F1']


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


0.8354919535262382 0.83365847640665 tensor(0.7662) tensor(0.7657) tensor(0.7781) tensor(0.7762) tensor(0.7617) tensor(0.7622) tensor(0.8259) tensor(0.8255)


### ANN with 1h/h average SBP & DBP 

In [4]:
data_range_pro = pd.read_csv('/content/drive/My Drive/hypertension test/Nocturnal_hypertension by C/data_10factors_gbk_dayBP_201222.csv', encoding = 'gbk', index_col = 0)
X_daytime = ['8-9SBP',
 '8-9DBP',
 '9-10SBP',
 '9-10DBP',
 '10-11SBP',
 '10-11DBP',
 '11-12SBP',
 '11-12DBP',
 '3-4SBP',
 '3-4DBP',
 '4-5SBP',
 '4-5DBP',
 '5-6SBP',
 '5-6DBP',
         
  '8-10SBP',
  '8-10DBP',
  '9-11SBP',
  '9-11DBP',
  '10-12SBP',
  '10-12DBP',
  '3-5SBP',
  '3-5DBP',
  '4-6SBP',
  '4-6DBP',
]            

In [8]:
df = pd.DataFrame( columns=('8-9SBP',
#  '8-9DBP',
 '9-10SBP',
#  '9-10DBP',
 '10-11SBP',
#  '10-11DBP',
 '11-12SBP',
#  '11-12DBP',
 '3-4SBP',
#  '3-4DBP',
 '4-5SBP',
#  '4-5DBP',
 '5-6SBP',
#  '5-6DBP',
         
  '8-10SBP',
#  '8-10DBP',
  '9-11SBP',
#  '9-11DBP',
  '10-12SBP',
#  '10-12DBP',
  '3-5SBP',
#  '3-5DBP',
  '4-6SBP',
#  '4-6DBP',
))

In [13]:
import torch
data_range_pro = pd.read_csv('/content/drive/My Drive/hypertension test/Nocturnal_hypertension by C/data_10factors_gbk_dayBP_201222.csv', encoding = 'gbk', index_col = 0)
def preprocess(data0):
    data = data0.copy()
    X_col = [col for col in data.columns if col != 'nocturnal_hypertension']
    dummy_col = [col for col in X_col if len(data[col].unique()) < 5]
    data[dummy_col] = data[dummy_col].astype('int64')
    float_col = [col for col in X_col if col not in dummy_col]
    # standard
    m,s = data[float_col].mean(), data[float_col].std()
    for col in float_col:
        data[col] = (data[col] - data[col].mean()) / (data[col].std())
    temp = pd.DataFrame()
    for col in dummy_col:
        temp = pd.concat([temp, pd.get_dummies(data[col], prefix = col, prefix_sep = '_', drop_first = True)], axis = 1)

    temp = pd.concat([data[float_col], temp, data['nocturnal_hypertension']], axis = 1)
    temp.index = pd.Index(range(len(temp)))
    return temp, m, s, float_col, dummy_col, X_col, 'nocturnal_hypertension'

data, m, s, float_col, dummy_col, X_col, y_col = preprocess(data_range_pro)
data['nocturnal_hypertension'] = data['nocturnal_hypertension'].astype('int')#bool转换成int

R = 115306

temp = ['Clinic SBP,mmHg','eGFR,mL/ (min·1.73 m2)','BUN,mmol/L','Clinic DBP,mmHg'
             ,'nRAAS drugs intake_1', 'Hypertension_1','Age,y'] + [y_col]

for j in range(int(len(X_daytime)/2)):
    i = j*2
    temp = ['Clinic SBP,mmHg','eGFR,mL/ (min·1.73 m2)','BUN,mmol/L','Clinic DBP,mmHg'
             ,'nRAAS drugs intake_1', 'Hypertension_1','Age,y',
      X_daytime[i],X_daytime[i+1]
    ] + [y_col]
    data_tensor = torch.from_numpy(data[temp].values).float()

    X = data_tensor[:,:-1]
    y = data_tensor[:,-1].resize(3103,1)

    kf = KFold(10,shuffle = True, random_state = R)

    result = []
    for train_idx, test_idx in kf.split(range(X.shape[0])):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        model = torch.nn.Sequential(torch.nn.Linear(9,15),
                                torch.nn.Sigmoid(), 
                                torch.nn.Linear(15,1),
                                torch.nn.Sigmoid())
        loss = torch.nn.MSELoss(reduce = False)
        optimizer=torch.optim.SGD(model.parameters(),lr=0.05,weight_decay=0.001)
        sw = y_train.clone()

        sw[sw == 1] = y_train.shape[0]/(2*y_train.float().sum())   
        
        sw[sw == 0] = y_train.shape[0]/(2*(y_train==0).float().sum())
        
        for epoch in range(10000):
            optimizer.zero_grad()  
            out = model(X_train)  
            loss1 = (sw * loss(out, y_train)).sum()/sw.sum()  
            loss1.backward()    
            optimizer.step()    
        out_train = model(X_train) 
        out_test = model(X_test)

        acc_train = ((out_train>0.5).float()==y_train).float().mean()
        acc_test = ((out_test>0.5).float()==y_test).float().mean()
        sen_train =  ((out_train >0.5).float() == y_train)[y_train == 1].float().mean()
        sen_test =  ((out_test >0.5).float() == y_test)[y_test == 1].float().mean()
        spe_train =  ((out_train >0.5).float() == y_train)[y_train == 0].float().mean()
        spe_test =  ((out_test >0.5).float() == y_test)[y_test == 0].float().mean()

        train_fpr, train_tpr, train_thresholds = roc_curve(y_train.detach().numpy(), out_train.detach().numpy())
        test_fpr, test_tpr, test_thresholds = roc_curve(y_test.detach().numpy(), out_test.detach().numpy())
        train_auc = auc(train_fpr, train_tpr)
        test_auc = auc(test_fpr, test_tpr)

        train_pre = ((out_train>0.5)&(y_train == 1)).sum().numpy()/(out_train >0.5).sum().numpy()
        test_pre = ((out_test>0.5)&(y_test == 1)).sum().numpy()/(out_test>0.5).sum().numpy()

        F1_train = (2*train_pre*sen_train)/(train_pre + sen_train)
        F1_test = (2*test_pre*sen_test)/(test_pre + sen_test)

        result.append({'train':{'acc':acc_train, 'sensitivity':sen_train, 'specivity':spe_train,'auc':train_auc, 'F1':F1_train},
                      'test':{'acc':acc_test, 'sensitivity':sen_test, 'specivity':spe_test,'auc':test_auc, 'F1':F1_test}})
    acc_train, acc_test, sen_train, sen_test, spe_train, spe_test, auc_train, auc_test,F1_train, F1_test = 0,0,0,0,0,0,0,0,0,0
    for temp in result:
        a =  temp['train']
        b = temp ['test']
        acc_train += 0.1*a['acc']
        acc_test += 0.1*b['acc']
        sen_train += 0.1*a['sensitivity']
        sen_test += 0.1*b['sensitivity']
        spe_train += 0.1*a['specivity']
        spe_test += 0.1*b['specivity']
        auc_train += 0.1*a['auc']
        auc_test += 0.1*b['auc']
        F1_train += 0.1*a['F1']
        F1_test += 0.1 * b['F1']

8-9SBP 8-9DBP


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


0.8718319730686783 0.8705224698895966 tensor(0.7929) tensor(0.7915) tensor(0.8043) tensor(0.8025) tensor(0.7886) tensor(0.7880) tensor(0.8472) tensor(0.8460)
9-10SBP 9-10DBP
0.8705949498809304 0.8701900861933819 tensor(0.7912) tensor(0.7915) tensor(0.8014) tensor(0.8006) tensor(0.7874) tensor(0.7885) tensor(0.8460) tensor(0.8462)
10-11SBP 10-11DBP
0.8728976370246769 0.8713112248613948 tensor(0.7873) tensor(0.7850) tensor(0.7956) tensor(0.7893) tensor(0.7842) tensor(0.7842) tensor(0.8430) tensor(0.8414)
11-12SBP 11-12DBP
0.8723007412106318 0.8701991786880264 tensor(0.7931) tensor(0.7915) tensor(0.7961) tensor(0.7936) tensor(0.7920) tensor(0.7911) tensor(0.8480) tensor(0.8466)
3-4SBP 3-4DBP
0.8789324543514843 0.8775116259703728 tensor(0.7955) tensor(0.7950) tensor(0.8111) tensor(0.8098) tensor(0.7896) tensor(0.7899) tensor(0.8490) tensor(0.8487)
4-5SBP 4-5DBP
0.8821795284020374 0.8800920318280515 tensor(0.7938) tensor(0.7915) tensor(0.8116) tensor(0.8071) tensor(0.7872) tensor(0.7858) te

### **Test with extra data**

In [21]:
data_range_pro = pd.read_csv('/content/drive/My Drive/hypertension test/Nocturnal_hypertension by C/data_10factors_gbk_dayBP_201222.csv', encoding = 'gbk', index_col = 0)
data_ex_valid = pd.read_csv('/content/drive/My Drive/hypertension test/Nocturnal_hypertension by C/data_ex_valid.csv', encoding = 'gbk', index_col = 0)
data_ex_valid.rename(columns = {'姓名':'Name','编号':'No.','性别（1男2女）':'Male','年龄':'Age,y','BMI':'BMI,kg/m2','吸烟0否1是':'Smoker'
                       ,'饮酒':'Alcohol intake','糖尿病':'Diabetes mellitus','高血压':'Hypertension','高脂血症':'Hyperlipidemia'
                       ,'高血压家族史':'Hypertension family history','CVD既往史':'CVD history','是否用降压药':'Antihypertension drugs'
                       ,'RAAS药':'RAAS drugs intake','nRAAS药':'nRAAS drugs intake','24H尿蛋白mg':'Proteinuria,mg/24h'
                       ,'HGB':'Hemoglobin,g/L','尿素氮':'BUN,mmol/L','肌酐':'Scr,mmol/L','尿酸':'Uric acid,mmol/L'
                       ,'空腹血糖':'FBG,mmol/L','甘油三酯':'Triglyceride,mmol/L','胆固醇':'Cholesterol,mmol/L','高密度脂蛋白':'HDL-C,mmol/L'
                       ,'低密度脂蛋白':'LDL-C,mmol/L','白蛋白':'Serum albumin,g/L','Na':'Serum sodium,mmol/L','CA':'Serum calcium,mmol/L'
                       ,'P':'Serum phosphate,mmol/L','B2-微球蛋白':'β2-MG,ug/mL','甲状旁腺素（pg/ml）':'iPTH,pg/mL'
                       ,'同型半胱氨酸':'Homocysteine,umol/L','GFR-EPI':'eGFR,mL/ (min·1.73 m2)'
                       ,'诊室SBP':'Clinic SBP,mmHg','诊室DBP':'Clinic DBP,mmHg'},inplace = True)
data_ex_valid = data_ex_valid[['Clinic SBP,mmHg','eGFR,mL/ (min·1.73 m2)','BUN,mmol/L','Clinic DBP,mmHg'
             ,'nRAAS drugs intake', 'Hypertension'
             ,'Age,y'
             ,'10-11DBP', '10-11SBP', '10-12DBP', '10-12SBP', '11-12DBP', '11-12SBP'
             ,'3-4DBP', '3-4SBP', '3-5DBP', '3-5SBP', '4-5DBP', '4-5SBP', '4-6DBP','4-6SBP'
             , '5-6DBP', '5-6SBP', '8-10DBP', '8-10SBP'
             , '8-9DBP', '8-9SBP', '9-10DBP', '9-10SBP', '9-11DBP','9-11SBP','nocturnal_hypertension']]

In [22]:
X_daytime = ['8-9SBP',
 '8-9DBP',
 '9-10SBP',
 '9-10DBP',
 '10-11SBP',
 '10-11DBP',
 '11-12SBP',
 '11-12DBP',
 '3-4SBP',
 '3-4DBP',
 '4-5SBP',
 '4-5DBP',
 '5-6SBP',
 '5-6DBP',
         
  '8-10SBP',
  '8-10DBP',
  '9-11SBP',
  '9-11DBP',
  '10-12SBP',
  '10-12DBP',
  '3-5SBP',
  '3-5DBP',
  '4-6SBP',
  '4-6DBP',
]

In [23]:
data, m, s, float_col, dummy_col, X_col, y_col = preprocess(data_range_pro)
data['nocturnal_hypertension'] = data['nocturnal_hypertension'].astype('int')#bool转换成int
data_ex, m_ex, s_ex, float_col_ex, dummy_col_ex, X_col_ex, y_col_ex = preprocess(data_ex_valid)
data_ex['nocturnal_hypertension'] = data_ex['nocturnal_hypertension'].astype('int')#bool转换成int                                                                                 

for j in range(int(len(X_daytime)/2)):
    i = j*2
    temp = ['Clinic SBP,mmHg','eGFR,mL/ (min·1.73 m2)','BUN,mmol/L','Clinic DBP,mmHg'
             ,'nRAAS drugs intake_1', 'Hypertension_1','Age,y',
      X_daytime[i],X_daytime[i+1]
    ] + [y_col]

    data_tensor = torch.from_numpy(data[temp].values).float()
    data_tensor_ex = torch.from_numpy(data_ex[temp].values).float()

    X = data_tensor[:,:-1]
    y = data_tensor[:,-1].resize(3103,1)

    X_ex = data_tensor_ex[:,:-1]
    y_ex = data_tensor_ex[:,-1].resize(229,1)

    kf = KFold(10,shuffle = True, random_state = 115306)

    result = []
    for train_idx, test_idx in kf.split(range(X.shape[0])):

        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        model = torch.nn.Sequential(torch.nn.Linear(9,15),
                                 torch.nn.Sigmoid(), 

                                 torch.nn.Linear(15,1),
                                 torch.nn.Sigmoid())

        loss = torch.nn.MSELoss(reduce = False)

        optimizer=torch.optim.SGD(model.parameters(),lr=0.05,weight_decay=0.001)

        sw = y_train.clone()

        sw[sw == 1] = y_train.shape[0]/(2*y_train.float().sum())   

        sw[sw == 0] = y_train.shape[0]/(2*(y_train==0).float().sum())

        for epoch in range(10000):
            optimizer.zero_grad()  
            out = model(X_train)   
            loss1 = (sw * loss(out, y_train)).sum()/sw.sum()   

            loss1.backward()     
            optimizer.step()     

        out_train = model(X_train) 
        out_test = model(X_test)

        acc_train = ((out_train>0.5).float()==y_train).float().mean()
        acc_test = ((out_test>0.5).float()==y_test).float().mean()

        sen_train =  ((out_train >0.5).float() == y_train)[y_train == 1].float().mean()
        sen_test =  ((out_test >0.5).float() == y_test)[y_test == 1].float().mean()

        spe_train =  ((out_train >0.5).float() == y_train)[y_train == 0].float().mean()
        spe_test =  ((out_test >0.5).float() == y_test)[y_test == 0].float().mean()

        train_fpr, train_tpr, train_thresholds = roc_curve(y_train.detach().numpy(), out_train.detach().numpy())
        test_fpr, test_tpr, test_thresholds = roc_curve(y_test.detach().numpy(), out_test.detach().numpy())
        train_auc = auc(train_fpr, train_tpr)
        test_auc = auc(test_fpr, test_tpr)

        train_pre = ((out_train>0.5)&(y_train == 1)).sum().numpy()/(out_train >0.5).sum().numpy()
        test_pre = ((out_test>0.5)&(y_test == 1)).sum().numpy()/(out_test>0.5).sum().numpy()

        F1_train = (2*train_pre*sen_train)/(train_pre + sen_train)
        F1_test = (2*test_pre*sen_test)/(test_pre + sen_test)


        result.append({'train':{'acc':acc_train, 'sensitivity':sen_train, 'specivity':spe_train,'auc':train_auc, 'F1':F1_train},
                       'test':{'acc':acc_test, 'sensitivity':sen_test, 'specivity':spe_test,'auc':test_auc, 'F1':F1_test}})

    acc_train, acc_test, sen_train, sen_test, spe_train, spe_test, auc_train, auc_test,F1_train, F1_test = 0,0,0,0,0,0,0,0,0,0
    for temp in result:
        a =  temp['train']
        b = temp ['test']

        acc_train += 0.1*a['acc']
        acc_test += 0.1*b['acc']
        sen_train += 0.1*a['sensitivity']
        sen_test += 0.1*b['sensitivity']
        spe_train += 0.1*a['specivity']
        spe_test += 0.1*b['specivity']
        auc_train += 0.1*a['auc']
        auc_test += 0.1*b['auc']
        F1_train += 0.1*a['F1']
        F1_test += 0.1 * b['F1']

    out_ex = model(X_ex)
    print(len(out_ex))
    acc_ex = ((out_ex>0.5).float()==y_ex).float().mean()
    sen_ex =  ((out_ex >0.5).float() == y_ex)[y_ex == 1].float().mean()
    spe_ex =  ((out_ex >0.5).float() == y_ex)[y_ex == 0].float().mean()

    test_fpr, test_tpr, test_thresholds = roc_curve(y_ex.detach().numpy(), out_ex.detach().numpy())
    auc_ex = auc(test_fpr, test_tpr)
    test_pre = ((out_ex>0.5)&(y_ex == 1)).sum().numpy()/(out_ex>0.5).sum().numpy()

    F1_ex = (2*test_pre*sen_test)/(test_pre + sen_test)


8-9SBP 8-9DBP


/usr/local/lib/python3.6/dist-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


0.8717950925385561 0.8704791523263262 tensor(0.7926) tensor(0.7918) tensor(0.8035) tensor(0.8049) tensor(0.7886) tensor(0.7876) tensor(0.8471) tensor(0.8461)
229
tensor(0.7642) tensor(0.7568) tensor(0.7778) 0.8293293293293293 tensor(0.8229)
9-10SBP 9-10DBP
0.8704921755126124 0.8699243754132548 tensor(0.7915) tensor(0.7908) tensor(0.8018) tensor(0.8018) tensor(0.7877) tensor(0.7872) tensor(0.8462) tensor(0.8456)
229
tensor(0.7555) tensor(0.7500) tensor(0.7654) 0.8303303303303304 tensor(0.8192)
10-11SBP 10-11DBP
0.872925838797769 0.8709792235817952 tensor(0.7872) tensor(0.7863) tensor(0.7952) tensor(0.7918) tensor(0.7843) tensor(0.7850) tensor(0.8430) tensor(0.8424)
229
tensor(0.7467) tensor(0.7432) tensor(0.7531) 0.8308308308308308 tensor(0.8145)
11-12SBP 11-12DBP
0.8725670257732832 0.870135136869445 tensor(0.7924) tensor(0.7915) tensor(0.7953) tensor(0.7926) tensor(0.7912) tensor(0.7916) tensor(0.8473) tensor(0.8467)
229
tensor(0.7948) tensor(0.7905) tensor(0.8025) 0.8502669336002668 t